# Data Preprocessing

In this Notebook, we will conduct initial preprocessing of the two JSON files we obtained in the previous Notebook. We will aim to create relevant Pandas DataFrames and save files as `csv` for subsequent exploration and modelling. In particular, we will extract user ids and game ids to create a user-item interactions DataFrame, with each row being a particular user-item relationship (namely owned item).

In [1]:
# Import libraries
import pandas as pd
import numpy as np

# Import Warnings
import warnings

warnings.filterwarnings("ignore")


## Games data

We will first load the `gamesdata` file, which has a row for each game and various descriptive features as columns.

In [2]:
# Load games data
df = pd.read_json("gamesdata.json")
df.sample(5)


,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
1761,NaN,[Action],The Bureau: XCOM Declassified - Hangar 6 R&D,The Bureau: XCOM Declassified - Hangar 6 R&amp;D,http://store.steampowered.com/app/66002/The_Bu...,2013-11-19,[Action],NaN,http://steamcommunity.com/app/66002/reviews/?b...,"[Single-player, Downloadable Content, Steam Ac...",4.99,False,66002.0,2K Marin,Mixed,NaN
18292,Paradox Interactive,"[Simulation, Strategy]",Collection - Europa Universalis IV: Early Upgr...,Collection - Europa Universalis IV: Early Upgr...,http://store.steampowered.com/app/617962/Colle...,2017-04-05,"[Strategy, Simulation]",NaN,http://steamcommunity.com/app/617962/reviews/?...,"[Single-player, Multi-player, Cross-Platform M...",19.99,False,617962.0,Paradox Development Studio,Mostly Negative,NaN
11957,NaN,"[Indie, RPG, Strategy]",Fantasy Grounds - Conquering Heroes (5E),Fantasy Grounds - Conquering Heroes (5E),http://store.steampowered.com/app/731140/Fanta...,2017-10-18,"[Strategy, RPG, Indie, Turn-Based, Fantasy, So...",NaN,http://steamcommunity.com/app/731140/reviews/?...,"[Multi-player, Co-op, Cross-Platform Multiplay...",5.99,False,731140.0,"SmiteWorks USA, LLC",NaN,NaN
23068,Running With Scissors,"[Action, Indie]",POSTAL Redux,POSTAL Redux,http://store.steampowered.com/app/401680/POSTA...,2016-05-20,"[Action, Gore, Indie, Violent, Isometric, Rema...",NaN,http://steamcommunity.com/app/401680/reviews/?...,"[Single-player, Online Multi-Player, Online Co...",8.99,False,401680.0,Running With Scissors,Very Positive,54
29590,NaN,"[Casual, Simulation]",Rocksmith® 2014 – Oasis - “Champagne Supernova”,Rocksmith® 2014 – Oasis - “Champagne Supernova”,http://store.steampowered.com/app/258415/Rocks...,2014-01-14,"[Casual, Simulation]",NaN,http://steamcommunity.com/app/258415/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",2.99,False,258415.0,Ubisoft - San Francisco,2 user reviews,NaN


We note that there are certain features which are lists, namely genres, tags and specs. These will be investigated further in the Data Exploration phase.

In [3]:
# Save as csv
df.to_csv("gamesdata.csv")


## User items Data

We now load the user items data, which has users as rows and details regarding items owned as columns.

In [4]:
# Load users/items data
useritems = pd.read_json("data.json")
useritems.head()


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Whilst the `items_count` feature will not be relevant for the user-item interactions/ recommendation engine, it will be relevant to our stakeholders for the purpose of understanding their user base. As such let us extract it and save it for the data exploration phase.

In [5]:
# Extract items_count feat
numgames = useritems[["user_id", "items_count"]]
numgames.head()


,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541


In [6]:
# Save as csv
numgames.to_csv("numgames.csv")


We note that the `items` column appears to be a list of dictionaries. Let us look at it in further detail.

In [7]:
# # Preview items column values for first user
# # Restrict to first 2 items in dictionary

# useritems["items"][0][0:2]


Each game is represented by a dictionary with keys the game's `item_id`, `item_name`, `playtime_forever` and `playtime_2weeks`. The dictionaries are then storred in a list.

We will look to extract the `item_id`'s into a seperate column. For now we will leave the playtime data but look to incorporate it later.

In [8]:
# Get all item_id's for first user
gameids = [
    useritems["items"][0][index]["item_id"]
    for index, _ in enumerate(useritems["items"][0])
]
# Show first 10 item ids
gameids[:10]


['10', '20', '30', '40', '50', '60', '70', '130', '300', '240']

Extract `item_id` from the dictionaries for each user.

In [9]:
# Create column with item ids
useritems["item_id"] = useritems["items"].apply(
    lambda x: [x[index]["item_id"] for index, _ in enumerate(x)]
)
useritems[["user_id", "items_count", "items", "item_id"]].head(10)
# useritems.head(10)


,user_id,items_count,items,item_id
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."
5,MinxIsBetterThanPotatoes,371,"[{'item_id': '50', 'item_name': 'Half-Life: Op...","[50, 240, 320, 4000, 6570, 220, 340, 360, 380,..."
6,NitemarePK,304,"[{'item_id': '240', 'item_name': 'Counter-Stri...","[240, 3830, 4000, 320, 340, 9180, 12900, 20, 5..."
7,themanwich,258,"[{'item_id': '220', 'item_name': 'Half-Life 2'...","[220, 340, 320, 360, 380, 4000, 6860, 7670, 40..."
8,maplemage,629,"[{'item_id': '240', 'item_name': 'Counter-Stri...","[240, 300, 320, 340, 1200, 1230, 1280, 2100, 2..."
9,Wackky,0,[],[]


As the unique user `steam_id` is a large integer, we will replace it with a new `uid` counter, which starts at 0 and increments by 1 (like the index).

We will also only select the relevant columns for the purpose of building a user-item interactions matrix, namely the newly created user id `uid` and the `item_id`.

In [10]:
# Add a column with substitute user_id, counter
useritems["uid"] = np.arange(len(useritems))

# Take relevant columns
useritems = useritems[["uid", "item_id"]]

# Check
useritems.head()


,uid,item_id
0,0,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,1,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,2,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,3,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,4,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


The next step is to explode the `item_id` into seperate rows, so each user-item interaction has it's own row.

In [11]:
# Explode item_ids into seperate rows
lst_col = "item_id"
useritems = pd.DataFrame(
    {
        col: np.repeat(useritems[col].values, useritems[lst_col].str.len())
        for col in useritems.columns.difference([lst_col])
    }
).assign(**{lst_col: np.concatenate(useritems[lst_col].values)})[
    useritems.columns.tolist()
]
useritems


,uid,item_id
0,0,10
1,0,20
2,0,30
3,0,40
4,0,50
...,...,...
2774489,29999,239140
2774490,29999,730
2774491,29999,355840
2774492,29999,366844


Add a binary column `owned` which will have 1s everywhere, as only items owned appear in the DataFrame.

In [12]:
# Add binary owned column
useritems["owned"] = np.ones(shape=useritems.shape[0])

# Check
useritems.head()


,uid,item_id,owned
0,0,10,1.0
1,0,20,1.0
2,0,30,1.0
3,0,40,1.0
4,0,50,1.0


In [13]:
len(useritems)


2774494

We note that we have individual user-item relationships represented in our DataFrame.

We want to restrict ourselves to user-item relationships where the item is in the first `gamesdata` DataFrame to be able to extract relevant information such as genre. 

We will ensure that the DataFrames can be merged on the game `id` feature by changing the type and column name.

In [14]:
# Change item_id to int
useritems["item_id"] = useritems["item_id"].astype(int)

# Rename column to match
useritems = useritems.rename(columns={"item_id": "id"})


We can now merge the DataFrames.

In [15]:
# Merge useritems and games data dataframes
alldata = pd.merge(useritems, df, on="id")
alldata.head()
    
alldata[["uid", "title", "genres", "release_date", "tags", "price", "metascore"]].head()

,uid,title,genres,release_date,tags,price,metascore
0,0,Counter-Strike,[Action],2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,88
1,1,Counter-Strike,[Action],2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,88
2,3,Counter-Strike,[Action],2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,88
3,4,Counter-Strike,[Action],2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,88
4,10,Counter-Strike,[Action],2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,88


In [16]:
len(alldata)


2327166

In [17]:
# Drop entries with no title
datawithnames = alldata.dropna(axis=0, subset=["title"])
datawithnames[
    ["uid", "genres", "title", "release_date", "tags", "specs", "price", "sentiment", "metascore"]
].sample(5)

# datawithnames.columns


,uid,genres,title,release_date,tags,specs,price,sentiment,metascore
2266547,6022,"[Action, Indie]",Pineapple Smash Crew,2012-02-02,"[Indie, Action, Rogue-like, Twin Stick Shooter]",[Single-player],9.99,Very Positive,62
1306846,7893,"[Action, Free to Play, Indie]",Loadout,2014-01-31,"[Free to Play, Third-Person Shooter, Action, G...","[Multi-player, Co-op, Steam Achievements, Stea...",Free to Play,Very Positive,72
634342,10305,[Action],DOOM,2016-05-12,"[FPS, Action, Gore, Demons, Shooter, First-Per...","[Single-player, Multi-player, Co-op, Steam Ach...",29.99,Very Positive,85
1518389,16956,"[Action, Free to Play, Indie, Massively Multip...",Robocraft,2017-08-24,"[Free to Play, Robots, Building, Multiplayer, ...","[Multi-player, Online Multi-Player, MMO, Onlin...",Free to Play,Mostly Positive,NaN
1634197,25282,[Action],Grand Theft Auto: Vice City,2003-05-13,"[Open World, Action, 1980s, Classic, Great Sou...","[Single-player, Partial Controller Support]",9.99,Very Positive,94


In [18]:
len(datawithnames)


2287374

We will save this DataFrame as a `csv` file to conduct data exploration and gain statistics.

In [19]:
# Save to csv
datawithnames.to_csv("mergeddata.csv")


Finally, let us extract the relevant columns for our user-item interactions matrix.

In [20]:
# Get relevant columns for recommendation engine
recdata = datawithnames[["uid", "id", "owned"]]
recdata.head()


,uid,id,owned
0,0,10,1.0
1,1,10,1.0
2,3,10,1.0
3,4,10,1.0
4,10,10,1.0


In [21]:
# Save to csv
recdata.to_csv("recdata.csv")
